<a href="https://colab.research.google.com/github/svetlanama/goit-np-hw-01/blob/main/NP_HW_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнє завдання 7: Класифікація спаму за допомогою наївного Баєса

## Мета роботи

У цій роботі ми будемо класифікувати електронні листи як спам або не спам, використовуючи наївний баєсівський класифікатор.

**Набір даних**: Email Spam Classification Dataset  
**Мітки**: 1 -> Spam, 0 -> Not Spam (Ham)

## Завдання

1. Завантажити та розпакувати набір даних
2. Імпортувати необхідні бібліотеки для обробки текстів
3. Прочитати дані та відібрати підвибірку
4. Візуалізувати розподіл повідомлень за класами
5. Застосувати методи обробки тексту (nltk)
6. Підготувати структури даних для тренування та тестування
7. Реалізувати алгоритм наївного Баєса
8. Проаналізувати якість класифікатора

## Крок 1-2: Завантаження та розпакування даних

In [ ]:
# Завантажуємо набір даних
!wget -O SpamEmailClassificationDataset.zip https://github.com/goitacademy/NUMERICAL-PROGRAMMING-IN-PYTHON/blob/main/SpamEmailClassificationDataset.zip?raw=true

In [ ]:
# Розпаковуємо файл
!unzip -o SpamEmailClassificationDataset.zip

## Крок 3: Імпорт бібліотек

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Завантажимо необхідні ресурси NLTK
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Стоп-слова
stop_words = set(stopwords.words('english'))

# Для візуалізації хмар слів
from wordcloud import WordCloud

# Налаштування для відображення графіків
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Крок 4: Завантаження та вибірка даних

Оригінальний набір містить 83448 записів. Для роботи відберемо приблизно 2000 записів з рівною кількістю спаму та не-спаму.

In [ ]:
# Читаємо дані
df = pd.read_csv('./combined_data.csv')

# Перевіряємо та конвертуємо тип колонки label
# Інколи label може бути прочитаний як string, тому конвертуємо у numeric
if df['label'].dtype == 'object':
    df['label'] = pd.to_numeric(df['label'], errors='coerce')
    print("⚠ Колонку 'label' конвертовано у числовий формат")

print(f"Повний розмір датасету: {df.shape}")
print(f"\nКолонки: {df.columns.tolist()}")
print(f"Тип даних label: {df['label'].dtype}")
print(f"\nПерші рядки:")
df.head()

In [ ]:
# Перевіряємо розподіл класів у повному датасеті
print("Розподіл класів у повному датасеті:")
print(df['label'].value_counts())
print(f"\nВідсоток спаму: {df['label'].mean() * 100:.2f}%")

In [ ]:
# Відбираємо збалансовану вибірку: по 1000 записів кожного класу
n_samples = 1000

spam_samples = df[df['label'] == 1].sample(n=n_samples, random_state=42)
ham_samples = df[df['label'] == 0].sample(n=n_samples, random_state=42)

# Об'єднуємо та перемішуємо
df = pd.concat([spam_samples, ham_samples]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Розмір вибірки: {df.shape}")
print(f"\nРозподіл класів у вибірці:")
print(df['label'].value_counts())

## Крок 5: Візуалізація розподілу повідомлень за класами

In [ ]:
# Створюємо дві візуалізації: гістограму та кругову діаграму
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гістограма
df['label'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Розподіл повідомлень за класами (Гістограма)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Клас (0 = Ham, 1 = Spam)', fontsize=12)
axes[0].set_ylabel('Кількість повідомлень', fontsize=12)
axes[0].set_xticklabels(['Ham (Not Spam)', 'Spam'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Додаємо значення на стовпчики
for i, v in enumerate(df['label'].value_counts().sort_index()):
    axes[0].text(i, v + 20, str(v), ha='center', fontsize=12, fontweight='bold')

# Кругова діаграма
labels = ['Ham (Not Spam)', 'Spam']
sizes = df['label'].value_counts().sort_index()
colors = ['#2ecc71', '#e74c3c']
explode = (0.05, 0.05)

axes[1].pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Розподіл повідомлень за класами (Pie Chart)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✓ Вибірка містить рівну кількість повідомлень обох класів")

## Крок 6: Обробка текстів за допомогою NLTK

Застосовуємо наступні методи:
- Приведення до нижнього регістру
- Видалення спеціальних символів (залишаємо тільки літери)
- Лематизація (приведення слів до словникової форми)
- Видалення стоп-слів
- Видалення повторів слів у повідомленні

In [ ]:
# Подивимось на приклади до обробки
print("Приклади повідомлень ДО обробки:\n")
print("SPAM:")
print(df[df['label'] == 1]['text'].iloc[0][:300])
print("\n" + "="*80 + "\n")
print("HAM:")
print(df[df['label'] == 0]['text'].iloc[0][:300])

In [ ]:
# Обробка тексту згідно з завданням
corpus = []
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

for document in df["text"]:
    # Видаляємо все крім букв та приводимо до нижнього регістру
    document = re.sub("[^a-zA-Z]", " ", document).lower()
    
    # Розбиваємо на слова
    document = document.split()
    
    # Лематизація та видалення стоп-слів
    document = [lemmatizer.lemmatize(word) for word in document if word not in stop_words]
    
    # Видаляємо дублікати слів (унікальні слова)
    document = list(set(document))
    
    # З'єднуємо назад у текст
    document = " ".join(document)
    corpus.append(document)

# Зберігаємо оброблений текст
df["text"] = corpus

print(f"✓ Обробка завершена! Оброблено {len(corpus)} документів")

In [ ]:
# Подивимось на приклади після обробки
print("Приклади повідомлень ПІСЛЯ обробки:\n")
print("SPAM:")
print(df[df['label'] == 1]['text'].iloc[0][:300])
print("\n" + "="*80 + "\n")
print("HAM:")
print(df[df['label'] == 0]['text'].iloc[0][:300])

## Крок 7: Підготовка структур даних для тренування

Створюємо структури:
- `train_spam` - список повідомлень-спаму для тренування
- `train_ham` - список не-спам повідомлень для тренування
- `test_emails` - словник тестових повідомлень з їх реальними мітками

In [ ]:
# Розділяємо дані на тренувальні та тестові (80/20)
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

print(f"Тренувальна вибірка: {train_df.shape[0]} повідомлень")
print(f"Тестова вибірка: {test_df.shape[0]} повідомлень")
print(f"\nРозподіл у тренувальній вибірці:")
print(train_df['label'].value_counts())
print(f"\nРозподіл у тестовій вибірці:")
print(test_df['label'].value_counts())

In [ ]:
# Створюємо структури даних для алгоритму наївного Баєса

# Тренувальні дані - розділяємо на два списки
train_spam = train_df[train_df['label'] == 1]['text'].tolist()
train_ham = train_df[train_df['label'] == 0]['text'].tolist()

# Тестові дані - словник {текст: мітка}
test_emails = dict(zip(test_df['text'], test_df['label']))

print(f"Структури даних створено:")
print(f"  - train_spam: {len(train_spam)} повідомлень")
print(f"  - train_ham: {len(train_ham)} повідомлень")
print(f"  - test_emails: {len(test_emails)} повідомлень")
print(f"\nПриклад структури train_spam (перше повідомлення):")
print(train_spam[0][:150])
print(f"\nПриклад структури test_emails (перші 3 елементи):")
for i, (text, label) in enumerate(list(test_emails.items())[:3]):
    print(f"{i+1}. Label: {label}, Text: {text[:100]}...")

## Крок 8: Реалізація алгоритму наївного Баєса

### Теоретичні основи

Наївний баєсівський класифікатор базується на теоремі Баєса:

$$P(spam|email) = \frac{P(email|spam) \times P(spam)}{P(email)}$$

Для класифікації порівнюємо:
- $P(spam|email)$ - ймовірність що це спам
- $P(ham|email)$ - ймовірність що це не спам

### Алгоритм:

1. **Побудова словника** - створюємо множину всіх унікальних слів
2. **Обчислення ймовірностей класів**: $P(spam)$ та $P(ham)$
3. **Обчислення умовних ймовірностей слів** з згладжуванням Лапласа:
   $$P(word|class) = \frac{count(word, class) + 1}{total\_words(class) + vocabulary\_size}$$
4. **Класифікація** - для кожного тестового листа обчислюємо:
   $$score(class) = \log P(class) + \sum_{word} \log P(word|class)$$

In [ ]:
# Крок 1: Побудова словника
def build_vocabulary(train_spam, train_ham):
    """Створюємо множину всіх унікальних слів із тренувальних даних"""
    vocabulary = set()
    
    # Додаємо слова зі спаму
    for email in train_spam:
        vocabulary.update(email.split())
    
    # Додаємо слова з ham
    for email in train_ham:
        vocabulary.update(email.split())
    
    return vocabulary

vocabulary = build_vocabulary(train_spam, train_ham)

print(f"Розмір словника: {len(vocabulary)} унікальних слів")
print(f"\nПриклади слів зі словника:")
print(list(vocabulary)[:20])

In [ ]:
# Крок 2: Обчислення апріорних ймовірностей P(spam) та P(ham)
total_emails = len(train_spam) + len(train_ham)
p_spam = len(train_spam) / total_emails
p_ham = len(train_ham) / total_emails

print(f"Апріорні ймовірності:")
print(f"  P(spam) = {p_spam:.4f} ({len(train_spam)}/{total_emails})")
print(f"  P(ham)  = {p_ham:.4f} ({len(train_ham)}/{total_emails})")

In [ ]:
# Крок 3: Підрахунок слів у кожному класі
def count_words_in_class(emails):
    """Підраховуємо частоту кожного слова в наборі листів"""
    word_counts = Counter()
    
    for email in emails:
        words = email.split()
        word_counts.update(words)
    
    return word_counts

spam_word_counts = count_words_in_class(train_spam)
ham_word_counts = count_words_in_class(train_ham)

# Загальна кількість слів у кожному класі
total_spam_words = sum(spam_word_counts.values())
total_ham_words = sum(ham_word_counts.values())

print(f"Статистика слів:")
print(f"  Всього слів у спамі: {total_spam_words:,}")
print(f"  Всього слів у ham: {total_ham_words:,}")
print(f"  Унікальних слів у спамі: {len(spam_word_counts):,}")
print(f"  Унікальних слів у ham: {len(ham_word_counts):,}")

print(f"\nТоп-10 найчастіших слів у СПАМІ:")
for word, count in spam_word_counts.most_common(10):
    print(f"  {word}: {count}")

print(f"\nТоп-10 найчастіших слів у HAM:")
for word, count in ham_word_counts.most_common(10):
    print(f"  {word}: {count}")

In [ ]:
# Крок 4: Обчислення умовних ймовірностей P(word|class) зі згладжуванням Лапласа
def calculate_word_probabilities(word_counts, total_words, vocab_size):
    """Обчислюємо P(word|class) для всіх слів зі згладжуванням Лапласа"""
    word_probs = {}
    
    for word, count in word_counts.items():
        # Згладжування Лапласа: додаємо 1 до лічильника і розмір словника до знаменника
        word_probs[word] = (count + 1) / (total_words + vocab_size)
    
    return word_probs

vocab_size = len(vocabulary)

spam_word_probs = calculate_word_probabilities(spam_word_counts, total_spam_words, vocab_size)
ham_word_probs = calculate_word_probabilities(ham_word_counts, total_ham_words, vocab_size)

# Ймовірність для невідомих слів (які не зустрічались у тренуванні)
unknown_word_prob_spam = 1 / (total_spam_words + vocab_size)
unknown_word_prob_ham = 1 / (total_ham_words + vocab_size)

print(f"Умовні ймовірності обчислено:")
print(f"  Словник спаму: {len(spam_word_probs):,} слів")
print(f"  Словник ham: {len(ham_word_probs):,} слів")
print(f"  P(unknown|spam) = {unknown_word_prob_spam:.10f}")
print(f"  P(unknown|ham) = {unknown_word_prob_ham:.10f}")

In [ ]:
# Крок 5: Функція класифікації
def classify_email(email, p_spam, p_ham, spam_word_probs, ham_word_probs, 
                   unknown_word_prob_spam, unknown_word_prob_ham):
    """
    Класифікує email як spam або ham.
    Повертає: ('spam' або 'ham', log_prob_spam, log_prob_ham)
    """
    words = email.split()
    
    # Починаємо з логарифму апріорних ймовірностей
    log_prob_spam = np.log(p_spam)
    log_prob_ham = np.log(p_ham)
    
    # Додаємо логарифми умовних ймовірностей для кожного слова
    for word in words:
        # Для spam
        if word in spam_word_probs:
            log_prob_spam += np.log(spam_word_probs[word])
        else:
            log_prob_spam += np.log(unknown_word_prob_spam)
        
        # Для ham
        if word in ham_word_probs:
            log_prob_ham += np.log(ham_word_probs[word])
        else:
            log_prob_ham += np.log(unknown_word_prob_ham)
    
    # Порівнюємо ймовірності
    if log_prob_spam > log_prob_ham:
        return 'spam', log_prob_spam, log_prob_ham
    else:
        return 'ham', log_prob_spam, log_prob_ham

print("Функція класифікації створена!")
print("\nТестуємо на прикладі:")

# Беремо перший тестовий email
test_email = list(test_emails.keys())[0]
true_label = test_emails[test_email]

prediction, log_p_spam, log_p_ham = classify_email(
    test_email, p_spam, p_ham, spam_word_probs, ham_word_probs,
    unknown_word_prob_spam, unknown_word_prob_ham
)

print(f"Email: {test_email[:100]}...")
print(f"Справжня мітка: {'spam' if true_label == 1 else 'ham'}")
print(f"Передбачення: {prediction}")
print(f"Log P(spam|email) = {log_p_spam:.2f}")
print(f"Log P(ham|email) = {log_p_ham:.2f}")

In [ ]:
# Класифікуємо всі тестові email
predictions = []
true_labels = []

for email, true_label in test_emails.items():
    prediction, _, _ = classify_email(
        email, p_spam, p_ham, spam_word_probs, ham_word_probs,
        unknown_word_prob_spam, unknown_word_prob_ham
    )
    
    predictions.append(1 if prediction == 'spam' else 0)
    true_labels.append(true_label)

predictions = np.array(predictions)
true_labels = np.array(true_labels)

print(f"✓ Класифіковано {len(predictions)} тестових повідомлень")

## Крок 9: Аналіз якості класифікатора

### Метрики оцінки:
- **Accuracy** (точність) - частка правильно класифікованих email
- **Precision** (прецизійність) - частка справжніх спамів серед передбачених як спам
- **Recall** (повнота) - частка знайдених спамів серед усіх справжніх спамів
- **F1-score** - гармонічне середнє precision та recall

In [ ]:
# Обчислюємо метрики
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print("=" * 60)
print("РЕЗУЛЬТАТИ КЛАСИФІКАЦІЇ")
print("=" * 60)
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"F1-score:  {f1:.4f} ({f1*100:.2f}%)")
print("=" * 60)

# Матриця помилок
cm = confusion_matrix(true_labels, predictions)

print(f"\nМатриця помилок:")
print(f"                 Predicted Ham  Predicted Spam")
print(f"Actual Ham       {cm[0,0]:^13}  {cm[0,1]:^14}")
print(f"Actual Spam      {cm[1,0]:^13}  {cm[1,1]:^14}")

# Візуалізація матриці помилок
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Ham', 'Spam'], 
            yticklabels=['Ham', 'Spam'],
            cbar_kws={'label': 'Кількість'},
            annot_kws={'fontsize': 16, 'fontweight': 'bold'})
plt.title('Матриця помилок (Confusion Matrix)', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Справжня мітка', fontsize=12)
plt.xlabel('Передбачена мітка', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Аналіз: які слова найбільш характерні для спаму?
# Обчислюємо відношення P(word|spam) / P(word|ham)

word_spam_ratios = []

for word in vocabulary:
    prob_spam = spam_word_probs.get(word, unknown_word_prob_spam)
    prob_ham = ham_word_probs.get(word, unknown_word_prob_ham)
    
    # Відношення ймовірностей
    ratio = prob_spam / prob_ham
    
    word_spam_ratios.append({
        'word': word,
        'prob_spam': prob_spam,
        'prob_ham': prob_ham,
        'ratio': ratio
    })

# Сортуємо за відношенням (найбільш характерні для спаму зверху)
word_spam_ratios_sorted = sorted(word_spam_ratios, key=lambda x: x['ratio'], reverse=True)

print("\n" + "="*80)
print("ТОП-20 СЛІВ З НАЙБІЛЬШОЮ ЙМОВІРНІСТЮ У СПАМІ")
print("=" * 80)
print(f"{'Слово':<20} {'P(word|spam)':<15} {'P(word|ham)':<15} {'Ratio (spam/ham)'}")
print("-" * 80)

for item in word_spam_ratios_sorted[:20]:
    print(f"{item['word']:<20} {item['prob_spam']:<15.10f} {item['prob_ham']:<15.10f} {item['ratio']:<15.2f}")

print("=" * 80)

print("\n" + "="*80)
print("ТОП-20 СЛІВ З НАЙБІЛЬШОЮ ЙМОВІРНІСТЮ У HAM (НЕ-СПАМІ)")
print("=" * 80)
print(f"{'Слово':<20} {'P(word|spam)':<15} {'P(word|ham)':<15} {'Ratio (ham/spam)'}")
print("-" * 80)

word_ham_ratios_sorted = sorted(word_spam_ratios, key=lambda x: x['ratio'])

for item in word_ham_ratios_sorted[:20]:
    print(f"{item['word']:<20} {item['prob_spam']:<15.10f} {item['prob_ham']:<15.10f} {1/item['ratio']:<15.2f}")

print("=" * 80)

In [ ]:
# Візуалізація: хмари слів для спаму та ham

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Хмара слів для SPAM
spam_text = ' '.join(train_spam)
wordcloud_spam = WordCloud(width=800, height=400, 
                           background_color='white',
                           colormap='Reds',
                           max_words=100,
                           relative_scaling=0.5).generate(spam_text)

axes[0].imshow(wordcloud_spam, interpolation='bilinear')
axes[0].set_title('Хмара слів: SPAM', fontsize=16, fontweight='bold', color='#e74c3c')
axes[0].axis('off')

# Хмара слів для HAM
ham_text = ' '.join(train_ham)
wordcloud_ham = WordCloud(width=800, height=400, 
                          background_color='white',
                          colormap='Greens',
                          max_words=100,
                          relative_scaling=0.5).generate(ham_text)

axes[1].imshow(wordcloud_ham, interpolation='bilinear')
axes[1].set_title('Хмара слів: HAM (Not Spam)', fontsize=16, fontweight='bold', color='#2ecc71')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Висновки

### Результати роботи:

1. **Набір даних**: 
   - Використано Email Spam Classification Dataset
   - Відібрано 2000 збалансованих записів (1000 spam + 1000 ham)
   - Розподіл: 80% тренування, 20% тестування

2. **Обробка тексту**:
   - Застосовано методи бібліотеки NLTK
   - Лематизація слів
   - Видалення стоп-слів
   - Видалення повторів у межах одного повідомлення

3. **Реалізація наївного Баєса**:
   - Алгоритм реалізовано з нуля (без sklearn)
   - Використано згладжування Лапласа для обробки невідомих слів
   - Логарифмічне масштабування для запобігання числовим помилкам

4. **Якість класифікації**:
   - Досягнуто високу точність на тестовій вибірці
   - Модель добре розрізняє spam від ham
   - Збалансовані метрики precision та recall

5. **Аналіз характерних слів**:
   - Виявлено слова з найбільшою ймовірністю у спамі
   - Слова пов'язані з маркетингом, грошима, акціями характерні для спаму
   - Слова пов'язані з роботою, проектами, комунікацією характерні для легітимних листів

6. **Візуалізація**:
   - Хмари слів наочно демонструють різницю у лексиці spam та ham
   - Матриця помилок показує розподіл правильних та неправильних класифікацій

### Переваги наївного Баєса:
- ✓ Простота реалізації
- ✓ Швидкість роботи
- ✓ Добре працює з текстовими даними
- ✓ Не потребує великого обсягу даних
- ✓ Інтерпретованість результатів

### Можливі покращення:
- Використання n-грам (біграми, тріграми) замість окремих слів
- TF-IDF ваги замість простих частот
- Збільшення розміру тренувальної вибірки
- Більш складна обробка тексту (видалення HTML, правильна обробка URL)
- Порівняння з іншими алгоритмами (SVM, Random Forest, нейронні мережі)